**4. Write a program to demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample.**

In [1]:
import numpy as np
import math
import csv

In [2]:
class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ""
        #def __str__(self):
        #return self.attribute

In [3]:
def sub(data, col, delete):
    dict = {}
    items = np.unique(data[:, col])
    count = np.zeros((items.shape[0], 1), dtype = np.int32)
    for x in range(items.shape[0]):
        for y in range(data.shape[0]):
            if data[y, col] == items[x]:
                count[x] += 1
                
    for x in range(items.shape[0]):
        dict[items[x]] = np.empty((int(count[x]), data.shape[1]), dtype = '|S32')
        pos = 0
        for y in range(data.shape[0]):
            if data[y,col] == items[x]:
                dict[items[x]][pos] = data[y]
                pos += 1
        if delete:
            dict[items[x]] = np.delete(dict[items[x]], col, 1)
    return items, dict

In [4]:
def entropy(s):
    items = np.unique(s)
    if items.size == 1:
        return 0
    counts = np.zeros((items.shape[0], 1))
    sums = 0
    for x in range(items.shape[0]):
        counts[x] = sum(s == items[x]) / (s.size * 1.0)
        
    for count in counts:
        sums += -1 * count * math.log(count, 2)
        
    return sums

In [5]:
def gain(data, col):
    items, dict = sub(data, col, delete = False)
    total_size = data.shape[0]
    entropies = np.zeros((items.shape[0], 1))
    intrinsic = np.zeros((items.shape[0], 1))

    for x in range((items.shape[0])):
        ratio = dict[items[x]].shape[0] / (total_size * 1.0)
        entropies[x] = ratio * entropy(dict[items[x]][:, -1])
        intrinsic[x] = ratio * math.log(ratio, 2)

    total_entropy = entropy(data[:, -1])    
    iv = -1 * sum(intrinsic)

    for x in range(entropies.shape[0]):
        total_entropy -= entropies[x]

    return (total_entropy / iv)

In [6]:
def create(data, metadata):
    if(np.unique(data[:, -1])).shape[0] == 1:
        node = Node("")
        node.answer = np.unique(data[:, -1])[0]
        return node
    
    gains = np.zeros((data.shape[1] -1, 1))
    for col in range(data.shape[-1] -1):
        gains[col] = gain(data, col)
        
    split = np.argmax(gains)    
    node = Node(metadata[split])
    metadata = np.delete(metadata, split, 0)
    items, dict = sub(data, split, delete = True)
    
    for x in range(items.shape[0]):
        child = create(dict[items[x]], metadata)
        node.children.append((items[x], child))
    return node

In [7]:
def empty(size):
    s = ""
    for x in range(size):
        s += ""
    return s

In [8]:
def print_tree(node, level):
    if node.answer != "":
        print(empty(level), node.answer)
        return
    print(empty(level), node.attribute)
    
    for value, n in node.children:
        print(empty(level + 1), value)
        print_tree(n, level + 2)

In [9]:
def read_data(filename):
    with open(filename, 'r') as csvfile:
        datareader = csv.reader(csvfile, delimiter = ',')
        headers = next(datareader)
        metadata = []
        traindata = []
        for name in headers:
            metadata.append(name)
        for row in datareader:
            traindata.append(row)
    return(metadata, traindata)

In [10]:
metadata, traindata = read_data("/content/drive/MyDrive/AIML Lab/Pg4DataSet.csv")
data = np.array(traindata)
node = create(data,metadata)
print_tree(node, 0)

 humidity
 high
 outlook
 b'overcast'
 b'yes'
 b'rain'
 b'no'
 b'sunny'
 b'no'
 normal
 wind
 b'strong'
 outlook
 b'overcast'
 b'yes'
 b'rain'
 b'no'
 b'sunny'
 b'yes'
 b'weak'
 b'yes'
 strong
 b'no'
 weak
 b'yes'
